In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Main script. 
The following script runs a quick demo optimization for one seed and few function 
iterations. This is not intended to reproduce the results of the paper, but 
simply to demonstrate the methods. 
The optimization parameters used in the full-scale experiments of the 
paper are included in this script and can be run upon changing the binary condition in 
line 113. 

"""


import sys
sys.path.append('src')
sys.path.append('ptreeopt')
from src import SB, SBsim
from src import *
from ptreeopt import PTreeOpt, MultiprocessingExecutor
import logging
import csv
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# set optimization parameters
class OptimizationParameters(object):
    def __init__(self):
        self.cores    = 36 # this value is used only in the full scale optimization
        self.nseeds   = 1
        self.nobjs    = 1
        self.drought_type = [87, 0.83, 2] # set drought type here

#create Result class
class Result():
    pass

#initialize action table
action = []
capacity = []
om = []
cx = []
param = []
lifetime = []
t_depl = []
action_type = []

In [3]:
df_actions_table = pd.read_csv("data/actions_table.csv")
df_actions_table.head()

,action,capacity,om,cx,t_depl,lifetime,action_type
0,nothing,0,0.00000,0.00000,1,1,0
1,SW200,200,1.86547,29.15983,120,40,0
2,SW300,300,2.59251,36.26095,120,40,0
3,SW400,400,3.30036,42.68769,120,40,0
4,SW500,500,3.99490,48.70583,120,40,0


In [4]:
action = df_actions_table['action'].values
capacity = df_actions_table['capacity'].values
om = df_actions_table['om'].values
cx = df_actions_table['cx'].values
t_depl = df_actions_table['t_depl'].values
lifetime = df_actions_table['lifetime'].values
action_type = [int(val) for val in df_actions_table['action_type'].values]

In [6]:
#categorizing each action name by action type into an array indexed by action type
action_name = [[], [], [], [], []]
i = 0
for act in action:
        action_name[action_type[i]].extend( [act] )
        i += 1

In [7]:
opt_par = OptimizationParameters()

In [ ]:
# define parameters for model and algorithm 
model = SB(opt_par, action_name, capacity, om, cx, t_depl, lifetime) 
algorithm = PTreeOpt(model.simulate,
                     feature_bounds=[[0, 35000],
                                     [-3, 3], [-3, 3],
                                     [0, 12100],[0, 12100],[0, 12100],
                                     [-20000, 20000],[-20000, 20000],[-20000, 20000],
                                     [0,800], [0, 800], [0, 25] ],

                     feature_names=['Surface Storage',
                                    'SRI 1y', 'SRI 3y',
                                    'allocation 1y','allocation 3y','allocation 5y',
                                    'delta storage 1y','delta storage 3y','delta storage 5y',
                                    'installed capacity', 'capacity under construction', 'curtailment'],

                     discrete_actions=True,
                     action_names=action_name,
                     mu=10, 
                     cx_prob=0.70,
                     population_size=10, #set this parameter to 100 for full scale optimization 
                     max_depth=3,
                     multiobj=False,
                     )